In [130]:
import pandas as pd
import os
import requests
from nba_api.stats.static import players
from nba_api.stats.endpoints import leaguestandingsv3
from bs4 import BeautifulSoup

In [2]:
cwd = os.getcwd()
cwd

'c:\\Ziyuan Sui\\Data Science Extra\\My Own Project\\All NBA Team Prediction\\2024'

## Get the histroical data (train)

Get the All-NBA team data from 1989 to 2023

In [3]:
dfs=[]
for i in range(1989,2012):
    url = "https://www.basketball-reference.com/awards/awards_"+str(i)+".html"
    df = pd.read_html(url)[2]
    new_columns = [col[1] for col in df.columns]
    # Set the new column names
    df.columns = new_columns
    df['year']=i
    df.dropna(subset='Pos',inplace=True)
    dfs.append(df)
    

In [4]:
df_1 = pd.concat(dfs,ignore_index=True)
df_1.to_csv(r'c:\Ziyuan Sui\Data Science Extra\My Own Project\All NBA Team Prediction\2024\award_1989_2011.csv',index=False)

In [ ]:
dfs=[]
for i in range(2012,2024):
    url = "https://www.basketball-reference.com/awards/awards_"+str(i)+".html"
    df = pd.read_html(url)[2]
    new_columns = [col[1] for col in df.columns]
    # Set the new column names
    df.columns = new_columns
    df['year']=i
    df.dropna(subset='Pos',inplace=True)
    dfs.append(df)
    

In [13]:
df_2 = pd.concat(dfs,ignore_index=True)
df_2.to_csv(r'c:\Ziyuan Sui\Data Science Extra\My Own Project\All NBA Team Prediction\2024\award_2012_2023.csv',index=False)

In [19]:
firstteam = pd.concat([df_1,df_2],ignore_index=True)
firstteam.to_csv(r'c:\Ziyuan Sui\Data Science Extra\My Own Project\All NBA Team Prediction\2024\firstteam.csv',index=False)

In [20]:
firstteam.head()

# Tm Pos           Player   Age   Tm  Pts Won  Pts Max  Share  1st Tm  \
0  1st   F      Karl Malone  25.0  UTA    425.0    425.0  1.000    85.0   
1  1st   G    Magic Johnson  29.0  LAL    423.0    425.0  0.995    84.0   
2  1st   G   Michael Jordan  25.0  CHI    423.0    425.0  0.995    84.0   
3  1st   F  Charles Barkley  25.0  PHI    412.0    425.0  0.969    80.0   
4  1st   C  Hakeem Olajuwon  26.0  HOU    379.0    425.0  0.892    66.0   

   2nd Tm  ...   TRB   AST  STL  BLK    FG%    3P%    FT%    WS  WS/48  year  
0     0.0  ...  10.7   2.7  1.8  0.9  0.519  0.313  0.766  15.2  0.233  1989  
1     NaN  ...   7.9  12.8  1.8  0.3  0.509  0.314  0.911  16.1  0.267  1989  
2     NaN  ...   8.0   8.0  2.9  0.8  0.538  0.276  0.850  19.8  0.292  1989  
3     NaN  ...  12.5   4.1  1.6  0.8  0.579  0.216  0.753  16.1  0.250  1989  
4     NaN  ...  13.5   1.8  2.6  3.4  0.508  0.000  0.696  12.4  0.197  1989  

[5 rows x 24 columns]

In [7]:
firstteam = pd.read_csv(r'c:\Ziyuan Sui\Data Science Extra\My Own Project\All NBA Team Prediction\2024\firstteam.csv')
player_name = firstteam.Player.unique()

Get NBA player ID

In [8]:
# Get all NBA players
nba_players = players.get_players()

# List of desired player names
desired_players = player_name


# Filter the players and get their IDs
filtered_players = [
    {"id": player["id"], "name": player["full_name"]}
    for player in nba_players
    if player["full_name"] in desired_players
]

# Create a DataFrame with the filtered players' data
players_df = pd.DataFrame(filtered_players)

# Display the DataFrame
print(players_df)

          id                 name
0        949  Shareef Abdur-Rahim
1        149        Michael Adams
2     203500         Steven Adams
3    1628389          Bam Adebayo
4      76017          Danny Ainge
..       ...                  ...
313      788         Kevin Willis
314     1897    Metta World Peace
315     1460         James Worthy
316     2397             Yao Ming
317  1629027           Trae Young

[318 rows x 2 columns]


In [9]:
ids = players_df.id.unique()

In [10]:
df2 = pd.merge(firstteam,players_df,left_on='Player',right_on='name',how='inner')

In [11]:
df2.drop(columns='name',inplace=True)

In [12]:
df2['season'] = df2['year'].apply(lambda x: f"{x-1}-{str(x)[-2:]}")

In [13]:
df2.head()

# Tm Pos       Player   Age   Tm  Pts Won  Pts Max  Share  1st Tm  2nd Tm  \
0  1st   F  Karl Malone  25.0  UTA    425.0    425.0  1.000    85.0     0.0   
1  1st   F  Karl Malone  26.0  UTA    456.0    460.0  0.991    90.0     NaN   
2  1st   F  Karl Malone  27.0  UTA    464.0    480.0  0.967    88.0     NaN   
3  1st   F  Karl Malone  28.0  UTA    466.0    480.0  0.971    91.0     NaN   
4  1st   F  Karl Malone  29.0  UTA    377.0    490.0  0.769    45.0     NaN   

   ...  STL  BLK    FG%    3P%    FT%    WS  WS/48  year   id   season  
0  ...  1.8  0.9  0.519  0.313  0.766  15.2  0.233  1989  252  1988-89  
1  ...  1.5  0.6  0.562  0.372  0.762  15.9  0.245  1990  252  1989-90  
2  ...  1.1  1.0  0.527  0.286  0.770  15.5  0.225  1991  252  1990-91  
3  ...  1.3  0.6  0.526  0.176  0.778  15.1  0.237  1992  252  1991-92  
4  ...  1.5  1.0  0.552  0.200  0.740  15.4  0.238  1993  252  1992-93  

[5 rows x 26 columns]

Get the NBA standing for each past season

In [14]:
start_season = 1989
end_season = 2023

# Initialize an empty DataFrame to store the standings
all_standings = pd.DataFrame()

# Loop through the seasons
for season_year in range(start_season, end_season + 1):
    # Format the season string
    season = f"{season_year}-{str(season_year + 1)[-2:]}"

    # Get the league standings for the current season
    standings = leaguestandingsv3.LeagueStandingsV3(season=season)

    # Convert the result to a DataFrame
    standings_df = standings.get_data_frames()[0]

    # Append the current season's standings to the all_standings DataFrame
    all_standings = all_standings.append(standings_df, ignore_index=True)

# Print the full DataFrame
print(all_standings)

C:\Users\jacks\AppData\Local\Temp\ipykernel_45488\858207671.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_standings = all_standings.append(standings_df, ignore_index=True)
C:\Users\jacks\AppData\Local\Temp\ipykernel_45488\858207671.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_standings = all_standings.append(standings_df, ignore_index=True)
C:\Users\jacks\AppData\Local\Temp\ipykernel_45488\858207671.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_standings = all_standings.append(standings_df, ignore_index=True)
C:\Users\jacks\AppData\Local\Temp\ipykernel_45488\858207671.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

     LeagueID SeasonID      TeamID      TeamCity       TeamName TeamSlug  \
0          00    21989  1610612747   Los Angeles         Lakers   lakers   
1          00    21989  1610612765       Detroit        Pistons  pistons   
2          00    21989  1610612755  Philadelphia          76ers   sixers   
3          00    21989  1610612759   San Antonio          Spurs    spurs   
4          00    21989  1610612741       Chicago          Bulls    bulls   
...       ...      ...         ...           ...            ...      ...   
1018       00    22023  1610612766     Charlotte        Hornets  hornets   
1019       00    22023  1610612759   San Antonio          Spurs    spurs   
1020       00    22023  1610612764    Washington        Wizards  wizards   
1021       00    22023  1610612757      Portland  Trail Blazers  blazers   
1022       00    22023  1610612765       Detroit        Pistons  pistons   

     Conference ConferenceRecord  PlayoffRank ClinchIndicator  ...  \
0          West  

C:\Users\jacks\AppData\Local\Temp\ipykernel_45488\858207671.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_standings = all_standings.append(standings_df, ignore_index=True)


In [16]:
all_standings['full_name'] =  all_standings['TeamCity'] + ' ' + all_standings['TeamName']

In [17]:
all_standings.to_csv(r"c:\Ziyuan Sui\Data Science Extra\My Own Project\All NBA Team Prediction\2024\standing.csv",index=False)

In [18]:
all_standings[['SeasonID','full_name','WinPCT']]

SeasonID               full_name  WinPCT
0       21989      Los Angeles Lakers   0.768
1       21989         Detroit Pistons   0.720
2       21989      Philadelphia 76ers   0.646
3       21989       San Antonio Spurs   0.683
4       21989           Chicago Bulls   0.671
...       ...                     ...     ...
1018    22023       Charlotte Hornets   0.256
1019    22023       San Antonio Spurs   0.268
1020    22023      Washington Wizards   0.183
1021    22023  Portland Trail Blazers   0.256
1022    22023         Detroit Pistons   0.171

[1023 rows x 3 columns]

In [114]:
standing = pd.read_csv(r"C:\Ziyuan Sui\MSBA\Winter23\Sports Analytics\Passion Project\standing.csv")

In [115]:
player_name = firstteam.Player.unique()

In [116]:
# Get all NBA players
nba_players = players.get_players()

# List of desired player names
desired_players = player_name


# Filter the players and get their IDs
filtered_players = [
    {"id": player["id"], "name": player["full_name"]}
    for player in nba_players
    if player["full_name"] in desired_players
]

# Create a DataFrame with the filtered players' data
players_df = pd.DataFrame(filtered_players)

# Display the DataFrame
print(players_df)

          id                 name
0        949  Shareef Abdur-Rahim
1        149        Michael Adams
2     203500         Steven Adams
3    1628389          Bam Adebayo
4      76017          Danny Ainge
..       ...                  ...
313      788         Kevin Willis
314     1897    Metta World Peace
315     1460         James Worthy
316     2397             Yao Ming
317  1629027           Trae Young

[318 rows x 2 columns]


In [117]:
ids = players_df.id.unique()
firstteam= pd.merge(firstteam,players_df,left_on='Player',right_on='name',how='inner')
firstteam.drop(columns='name',inplace=True)
firstteam['season'] = firstteam['year'].apply(lambda x: f"{x-1}-{str(x)[-2:]}")
firstteam['SeasonID'] = firstteam['year'].apply(lambda x: f"2{x}")
firstteam=firstteam.sort_values(by = ['season', 'Share'], ascending = [True, False])

In [118]:
firstteam.head()

# Tm Pos           Player   Age   Tm  Pts Won  Pts Max  Share  1st Tm  \
0   1st   F      Karl Malone  25.0  UTA    425.0    425.0  1.000    85.0   
15  1st   G    Magic Johnson  29.0  LAL    423.0    425.0  0.995    84.0   
19  1st   G   Michael Jordan  25.0  CHI    423.0    425.0  0.995    84.0   
30  1st   F  Charles Barkley  25.0  PHI    412.0    425.0  0.969    80.0   
41  1st   C  Hakeem Olajuwon  26.0  HOU    379.0    425.0  0.892    66.0   

    2nd Tm  ...  BLK    FG%    3P%    FT%    WS  WS/48  year     id   season  \
0      0.0  ...  0.9  0.519  0.313  0.766  15.2  0.233  1989    252  1988-89   
15     NaN  ...  0.3  0.509  0.314  0.911  16.1  0.267  1989  77142  1988-89   
19     NaN  ...  0.8  0.538  0.276  0.850  19.8  0.292  1989    893  1988-89   
30     NaN  ...  0.8  0.579  0.216  0.753  16.1  0.250  1989    787  1988-89   
41     NaN  ...  3.4  0.508  0.000  0.696  12.4  0.197  1989    165  1988-89   

    SeasonID  
0      21989  
15     21989  
19     21989  
30     21989  
41     21989  

[5 rows x 27 columns]

In [119]:
dict_teams = {'Utah Jazz':'UTA','Phoenix Suns':'PHO',
                'Philadelphia 76ers':'PHI','Brooklyn Nets':'BRK',
                'Denver Nuggets':'DEN','Los Angeles Clippers':'LAC',
                'Milwaukee Bucks':'MIL','Dallas Mavericks':'DAL',
                'Los Angeles Lakers':'LAL','Portland Trail Blazers':'POR',
                'Atlanta Hawks':'ATL','New York Knicks':'NYK',
                'Miami Heat':'MIA','Golden State Warriors':'GSW',
                'Memphis Grizzlies':'MEM','Boston Celtics':'BOS',
                'Washington Wizards':'WAS','Indiana Pacers':'IND',
                'Charlotte Hornets':'CHO','Charlotte Bobcats':'CHA',
                'San Antonio Spurs':'SAS','Chicago Bulls':'CHI',
                'New Orleans Pelicans':'NOP','Sacramento Kings':'SAC',
                'Toronto Raptors':'TOR','Minnesota Timberwolves':'MIN',
                'Cleveland Cavaliers':'CLE','Oklahoma City Thunder':'OKC',
                'Orlando Magic':'ORL','Detroit Pistons':'DET',
                'Houston Rockets':'HOU','New Jersey Nets':'NJN',
                'New Orleans Hornets':'NOH','Seattle SuperSonics':'SEA','Vancouver Grizzlies':'VAN',
                'New Orleans/Oklahoma City Hornets':'NOK','Washington Bullets':'WSB','Charlotte Hornets':'CHH'}

teams = pd.DataFrame.from_dict(dict_teams, orient='index').reset_index()
teams = teams.rename(columns={'index':'Team',0:'Tm'})

In [120]:
df22 = firstteam.merge(teams,on='Tm')
df22=df22.sort_values(by = ['season', 'Share'], ascending = [True, False])

In [121]:
std = standing[['SeasonID','full_name','WinPCT']]
std=std.rename(columns={'full_name':'Team'})
df22.SeasonID = df22.SeasonID.astype(int)

In [123]:
df3 = pd.merge(df22,std,on=['SeasonID','Team'])


In [124]:
df3.columns

Index(['# Tm', 'Pos', 'Player', 'Age', 'Tm', 'Pts Won', 'Pts Max', 'Share',
       '1st Tm', '2nd Tm', '3rd Tm', 'G', 'MP', 'PTS', 'TRB', 'AST', 'STL',
       'BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48', 'year', 'id', 'season',
       'SeasonID', 'Team', 'WinPCT'],
      dtype='object')

Get the player link on basketball-reference.com

In [51]:
# List of years to scrape
y1 = [2017,2018,2019,2020,2021,2022,2023]
y2 = [2010,2011,2012,2013,2014,2015,2016]
y3 = [2003,2004,2005,2006,2007,2008,2009]
y4 = [1996,1997,1998,1999,2000,2001,2002]
y5 = [1989,1990,1991,1992,1993,1994,1995]
years = [y1,y2,y3,y4,y5]
# Initialize an empty list to store the player data
player_data = []

# Loop through the years
for i in range(len(years)):
    for year in years[i]:
        url = "https://www.basketball-reference.com/awards/awards_"+str(year)+".html"

        response = requests.get(url)
        content = response.content
        soup = BeautifulSoup(content, 'html.parser')

        all_nba_table = soup.find('table', {'id': 'leading_all_nba'})

        rows = all_nba_table.find_all('tr')

        # Loop through the rows
        for row in rows:
            cols = row.find_all('td')

            # Loop through the columns
            for col in cols:
                link = col.find('a')
                if link:
                    player_name = link.text
                    player_href = link['href']
                    player_data.append((year, player_name, player_href))

    # Create a DataFrame with the player data
    link_df = pd.DataFrame(player_data, columns=['Year', 'Player', 'Link'])

    # Print the DataFrame
    print(link_df)


AttributeError: 'NoneType' object has no attribute 'find_all'

In [42]:
link_final = pd.read_csv(r"c:\Ziyuan Sui\Data Science Extra\My Own Project\All NBA Team Prediction\2024\link_final.csv")

In [43]:
link_final.shape

(1432, 4)

In [39]:
#link_final = link_final[link_final['Link'].str.startswith('/players/')]
#link_final['Link'] = 'https://www.basketball-reference.com' + link_final['Link']
#link_final['season'] = link_final['Year'].apply(lambda x: f"{x-1}-{str(x)[-2:]}")
#link_final = link_final.sort_values('Year',ascending=True)

In [45]:
link_final

Year                   Player  \
0     2023    Giannis Antetokounmpo   
1     2023             Jayson Tatum   
2     2023              Joel Embiid   
3     2023  Shai Gilgeous-Alexander   
4     2023              Luka Dončić   
...    ...                      ...   
1427  1989            Eddie Johnson   
1428  1989          Xavier McDaniel   
1429  1989               Ken Norman   
1430  1989               Doc Rivers   
1431  1989           Kelly Tripucka   

                                                   Link   season  
0     https://www.basketball-reference.com/players/a...  2022-23  
1     https://www.basketball-reference.com/players/t...  2022-23  
2     https://www.basketball-reference.com/players/e...  2022-23  
3     https://www.basketball-reference.com/players/g...  2022-23  
4     https://www.basketball-reference.com/players/d...  2022-23  
...                                                 ...      ...  
1427  https://www.basketball-reference.com/players/j...  1988-89  
1428  https://www.basketball-reference.com/players/m...  1988-89  
1429  https://www.basketball-reference.com/players/n...  1988-89  
1430  https://www.basketball-reference.com/players/r...  1988-89  
1431  https://www.basketball-reference.com/players/t...  1988-89  

[1432 rows x 4 columns]

In [93]:
url= "https://www.basketball-reference.com/leagues/NBA_2023_advanced.html"
df=pd.read_html(url)[0]
df['year'] = 2023

In [98]:
df

Rk            Player Pos Age   Tm   G    MP   PER   TS%  3PAr  ...  OWS  \
0      1  Precious Achiuwa   C  23  TOR  55  1140  15.2  .554  .267  ...  0.8   
1      2      Steven Adams   C  29  MEM  42  1133  17.5  .564  .004  ...  1.3   
2      3       Bam Adebayo   C  25  MIA  75  2598  20.1  .592  .011  ...  3.6   
3      4      Ochai Agbaji  SG  22  UTA  59  1209   9.5  .561  .591  ...  0.9   
4      5      Santi Aldama  PF  22  MEM  77  1682  13.9  .591  .507  ...  2.1   
..   ...               ...  ..  ..  ...  ..   ...   ...   ...   ...  ...  ...   
700  535    Thaddeus Young  PF  34  TOR  54   795  14.1  .573  .172  ...  0.7   
701  536        Trae Young  PG  24  ATL  73  2541  22.0  .573  .331  ...  5.3   
702  537    Omer Yurtseven   C  24  MIA   9    83  16.7  .675  .259  ...  0.2   
703  538       Cody Zeller   C  30  MIA  15   217  16.4  .659  .034  ...  0.4   
704  539       Ivica Zubac   C  25  LAC  76  2170  16.7  .661  .004  ...  3.8   

     DWS   WS WS/48 Unnamed: 24  OBPM  DBPM   BPM  VORP  year  
0    1.4  2.2  .093         NaN  -1.4  -0.8  -2.3  -0.1  2023  
1    2.1  3.4  .144         NaN  -0.3   0.9   0.6   0.7  2023  
2    3.8  7.4  .137         NaN   0.8   0.8   1.5   2.3  2023  
3    0.4  1.3  .053         NaN  -1.7  -1.4  -3.0  -0.3  2023  
4    2.4  4.6  .130         NaN  -0.3   0.8   0.5   1.1  2023  
..   ...  ...   ...         ...   ...   ...   ...   ...   ...  
700  1.1  1.8  .109         NaN  -1.8   1.9   0.1   0.4  2023  
701  1.4  6.7  .126         NaN   5.3  -2.0   3.3   3.4  2023  
702  0.1  0.3  .159         NaN  -2.5  -1.5  -3.9   0.0  2023  
703  0.3  0.7  .147         NaN  -2.0  -0.7  -2.8   0.0  2023  
704  2.9  6.7  .149         NaN  -1.1   0.2  -0.9   0.6  2023  

[705 rows x 30 columns]

In [91]:
dfs1=[]
for i in range(2004,2023):
    url= "https://www.basketball-reference.com/leagues/NBA_"+str(i)+"_advanced.html"
    df=pd.read_html(url)[0]
    df['year'] = i
    dfs1.append(df)
result = pd.concat(dfs, ignore_index=True)

In [88]:
dfs2=[]
for i in range(1989,2004):
    url= "https://www.basketball-reference.com/leagues/NBA_"+str(i)+"_advanced.html"
    df=pd.read_html(url)[0]
    df['year'] = i
    dfs2.append(df)
result2 = pd.concat(dfs2, ignore_index=True)

In [95]:
result.year.max()

2022

In [96]:
result2

Rk                Player Pos Age   Tm   G    MP   PER   TS%  3PAr  ...  \
0       1  Kareem Abdul-Jabbar*   C  41  LAL  74  1695  12.9  .511  .005  ...   
1       2            Mark Acres   C  26  BOS  62   632   8.2  .507  .009  ...   
2       3         Michael Adams  PG  26  DEN  77  2787  17.5  .567  .431  ...   
3       4          Mark Aguirre  SF  29  TOT  80  2597  16.4  .524  .137  ...   
4       4          Mark Aguirre  SF  29  DAL  44  1529  17.3  .509  .119  ...   
...   ...                   ...  ..  ..  ...  ..   ...   ...   ...   ...  ...   
7665  424          Qyntel Woods  PF  21  POR  53   334  11.2  .505  .076  ...   
7666  425     Metta World Peace  SF  23  IND  69  2317  17.6  .529  .249  ...   
7667  426       Lorenzen Wright   C  27  MEM  70  1982  14.4  .489  .007  ...   
7668  427     Vincent Yarbrough  SG  21  DEN  59  1381   9.5  .446  .182  ...   
7669  428           Wang Zhizhi   C  25  LAC  41   412  11.6  .485  .290  ...   

       OWS  DWS   WS WS/48 Unnamed: 24  OBPM  DBPM   BPM  VORP  year  
0      0.9  2.0  2.9  .082         NaN  -1.9  -0.2  -2.1   0.0  1989  
1      0.4  0.5  0.9  .066         NaN  -3.0  -0.4  -3.4  -0.2  1989  
2      5.4  2.9  8.3  .143         NaN   2.7   0.2   2.9   3.4  1989  
3      2.4  2.1  4.5  .082         NaN   1.4  -1.3   0.1   1.4  1989  
4      0.7  1.0  1.7  .052         NaN   1.5  -1.9  -0.4   0.6  1989  
...    ...  ...  ...   ...         ...   ...   ...   ...   ...   ...  
7665  -0.1  0.4  0.2  .034         NaN  -3.4  -0.7  -4.1  -0.2  2003  
7666   3.2  4.1  7.2  .150         NaN   0.9   2.3   3.2   3.1  2003  
7667   1.0  1.5  2.5  .061         NaN  -1.6  -1.4  -3.0  -0.5  2003  
7668  -0.7  1.7  1.0  .035         NaN  -3.3   0.9  -2.4  -0.1  2003  
7669  -0.1  0.2  0.1  .014         NaN  -1.6  -2.2  -3.8  -0.2  2003  

[7670 rows x 30 columns]

In [99]:
advanced = pd.concat([df,result,result2])

In [100]:
advanced['Player'] = advanced['Player'].str.replace('*', '')
advanced['Player'] = advanced['Player'].str.replace('+', '')

C:\Users\jacks\AppData\Local\Temp\ipykernel_45488\1321753709.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  advanced['Player'] = advanced['Player'].str.replace('*', '')
C:\Users\jacks\AppData\Local\Temp\ipykernel_45488\1321753709.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  advanced['Player'] = advanced['Player'].str.replace('+', '')


In [101]:
advanced.to_csv(r"c:\Ziyuan Sui\Data Science Extra\My Own Project\All NBA Team Prediction\2024\advanced2.csv",index=False)

In [102]:
advanced = pd.read_csv(r"c:\Ziyuan Sui\Data Science Extra\My Own Project\All NBA Team Prediction\2024\advanced2.csv")

In [108]:
advanced.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'BPM', 'VORP', 'year'],
      dtype='object')

In [109]:
advanced = advanced.drop(columns=['Pos','Age','Tm','G','MP','WS','WS/48','Unnamed: 24','Unnamed: 19','Rk'])

In [125]:
df4 = pd.merge(df3,advanced,on=['Player','year'],how='left')

In [126]:
df4 = df4.drop(columns=['Pts Won', 'Pts Max', '1st Tm', '2nd Tm', '3rd Tm','id'])

In [128]:
df4.shape

(1317, 41)

In [129]:
#get the training data
df4.to_csv(r"C:\Ziyuan Sui\Data Science Extra\My Own Project\All NBA Team Prediction\2024\train2024.csv",index=False)

## Get the most recent season data (test)

Standing

In [55]:
# Set the desired season
season = '2023-24'

# Get the league standings
standings = leaguestandingsv3.LeagueStandingsV3(season=season)

# Convert the result to a DataFrame
standings_df = standings.get_data_frames()[0]

# Print the DataFrame
print(standings_df)

   LeagueID SeasonID      TeamID       TeamCity       TeamName      TeamSlug  \
0        00    22023  1610612738         Boston        Celtics       celtics   
1        00    22023  1610612760  Oklahoma City        Thunder       thunder   
2        00    22023  1610612743         Denver        Nuggets       nuggets   
3        00    22023  1610612752       New York         Knicks        knicks   
4        00    22023  1610612749      Milwaukee          Bucks         bucks   
5        00    22023  1610612750      Minnesota   Timberwolves  timberwolves   
6        00    22023  1610612739      Cleveland      Cavaliers     cavaliers   
7        00    22023  1610612746             LA       Clippers      clippers   
8        00    22023  1610612742         Dallas      Mavericks     mavericks   
9        00    22023  1610612753        Orlando          Magic         magic   
10       00    22023  1610612754        Indiana         Pacers        pacers   
11       00    22023  1610612756        

In [56]:
standings_df['full_name'] =  standings_df['TeamCity'] + ' ' + standings_df['TeamName']

In [57]:
std = standings_df[['full_name','WinPCT']]

In [58]:
std

full_name  WinPCT
0           Boston Celtics   0.780
1    Oklahoma City Thunder   0.695
2           Denver Nuggets   0.695
3          New York Knicks   0.610
4          Milwaukee Bucks   0.598
5   Minnesota Timberwolves   0.683
6      Cleveland Cavaliers   0.585
7              LA Clippers   0.622
8         Dallas Mavericks   0.610
9            Orlando Magic   0.573
10          Indiana Pacers   0.573
11            Phoenix Suns   0.598
12    New Orleans Pelicans   0.598
13      Philadelphia 76ers   0.573
14      Los Angeles Lakers   0.573
15              Miami Heat   0.561
16           Chicago Bulls   0.476
17        Sacramento Kings   0.561
18           Atlanta Hawks   0.439
19   Golden State Warriors   0.561
20         Houston Rockets   0.500
21           Brooklyn Nets   0.390
22         Toronto Raptors   0.305
23               Utah Jazz   0.378
24       Memphis Grizzlies   0.329
25       Charlotte Hornets   0.256
26       San Antonio Spurs   0.268
27      Washington Wizards   0.183
28  Portland Trail Blazers   0.256
29         Detroit Pistons   0.171

In [78]:
test = pd.read_csv(r"C:\Ziyuan Sui\Data Science Extra\My Own Project\All NBA Team Prediction\2024\test2024.csv")

In [79]:
test.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PTS', 'TRB', 'AST', 'STL',
       'BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48', 'PER', 'TS%', '3PAr', 'FTr',
       'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS',
       'DWS', 'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [80]:
dict_teams = {'Utah Jazz':'UTA','Phoenix Suns':'PHO',
                'Philadelphia 76ers':'PHI','Brooklyn Nets':'BRK',
                'Denver Nuggets':'DEN','LA Clippers':'LAC',
                'Milwaukee Bucks':'MIL','Dallas Mavericks':'DAL',
                'Los Angeles Lakers':'LAL','Portland Trail Blazers':'POR',
                'Atlanta Hawks':'ATL','New York Knicks':'NYK',
                'Miami Heat':'MIA','Golden State Warriors':'GSW',
                'Memphis Grizzlies':'MEM','Boston Celtics':'BOS',
                'Washington Wizards':'WAS','Indiana Pacers':'IND',
                'Charlotte Hornets':'CHO','Charlotte Bobcats':'CHA',
                'San Antonio Spurs':'SAS','Chicago Bulls':'CHI',
                'New Orleans Pelicans':'NOP','Sacramento Kings':'SAC',
                'Toronto Raptors':'TOR','Minnesota Timberwolves':'MIN',
                'Cleveland Cavaliers':'CLE','Oklahoma City Thunder':'OKC',
                'Orlando Magic':'ORL','Detroit Pistons':'DET',
                'Houston Rockets':'HOU','New Jersey Nets':'NJN',
                'New Orleans Hornets':'NOH','Seattle SuperSonics':'SEA','Vancouver Grizzlies':'VAN',
                'New Orleans/Oklahoma City Hornets':'NOK','Washington Bullets':'WSB','Charlotte Hornets':'CHH'}
teams = pd.DataFrame.from_dict(dict_teams, orient='index').reset_index()
teams = teams.rename(columns={'index':'Team',0:'Tm'})

In [81]:
teams.head()

Team   Tm
0           Utah Jazz  UTA
1        Phoenix Suns  PHO
2  Philadelphia 76ers  PHI
3       Brooklyn Nets  BRK
4      Denver Nuggets  DEN

In [82]:
teams = pd.merge(teams,std,left_on='Team',right_on='full_name')
teams.sort_values('Team',ascending=True)

Team   Tm               full_name  WinPCT
10           Atlanta Hawks  ATL           Atlanta Hawks   0.439
15          Boston Celtics  BOS          Boston Celtics   0.780
3            Brooklyn Nets  BRK           Brooklyn Nets   0.390
18       Charlotte Hornets  CHH       Charlotte Hornets   0.256
20           Chicago Bulls  CHI           Chicago Bulls   0.476
25     Cleveland Cavaliers  CLE     Cleveland Cavaliers   0.585
7         Dallas Mavericks  DAL        Dallas Mavericks   0.610
4           Denver Nuggets  DEN          Denver Nuggets   0.695
28         Detroit Pistons  DET         Detroit Pistons   0.171
13   Golden State Warriors  GSW   Golden State Warriors   0.561
29         Houston Rockets  HOU         Houston Rockets   0.500
17          Indiana Pacers  IND          Indiana Pacers   0.573
5              LA Clippers  LAC             LA Clippers   0.622
8       Los Angeles Lakers  LAL      Los Angeles Lakers   0.573
14       Memphis Grizzlies  MEM       Memphis Grizzlies   0.329
12              Miami Heat  MIA              Miami Heat   0.561
6          Milwaukee Bucks  MIL         Milwaukee Bucks   0.598
24  Minnesota Timberwolves  MIN  Minnesota Timberwolves   0.683
21    New Orleans Pelicans  NOP    New Orleans Pelicans   0.598
11         New York Knicks  NYK         New York Knicks   0.610
26   Oklahoma City Thunder  OKC   Oklahoma City Thunder   0.695
27           Orlando Magic  ORL           Orlando Magic   0.573
2       Philadelphia 76ers  PHI      Philadelphia 76ers   0.573
1             Phoenix Suns  PHO            Phoenix Suns   0.598
9   Portland Trail Blazers  POR  Portland Trail Blazers   0.256
22        Sacramento Kings  SAC        Sacramento Kings   0.561
19       San Antonio Spurs  SAS       San Antonio Spurs   0.268
23         Toronto Raptors  TOR         Toronto Raptors   0.305
0                Utah Jazz  UTA               Utah Jazz   0.378
16      Washington Wizards  WAS      Washington Wizards   0.183

In [83]:
test = pd.merge(test,teams,on='Tm')

In [84]:
position_mapping = {
    'PG': 'G',
    'SG': 'G',
    'SF': 'F',
    'PF': 'F',
    'C': 'C' }

# Apply the mapping
test['Pos'] = test['Pos'].replace(position_mapping)

In [85]:
test = test.drop(columns=['Team','full_name','Age','Tm'])

In [86]:
test.head()

Player Pos   G    MP   PTS  TRB  AST  STL  BLK    FG%  ...  BLK%  \
0  Precious Achiuwa   C  25  17.5   7.7  5.4  1.8  0.6  0.5  0.459  ...   2.4   
1      Ochai Agbaji   G  27  23.6   6.7  3.3  1.3  0.7  0.6  0.391  ...   2.1   
2        OG Anunoby   F  27  33.3  15.1  3.9  2.7  1.0  0.5  0.489  ...   1.3   
3    Scottie Barnes   G  60  34.9  19.9  8.2  6.1  1.3  1.5  0.475  ...   3.7   
4        RJ Barrett   G  32  33.5  21.8  6.4  4.1  0.6  0.4  0.553  ...   1.1   

   TOV%  USG%  OWS  DWS  OBPM  DBPM  BPM  VORP  WinPCT  
0  13.3  21.2  0.0  0.4  -1.4  -0.2 -1.6   0.0   0.305  
1  13.1  15.4 -0.6  0.3  -5.0  -1.5 -6.5  -0.7   0.305  
2  11.2  18.2  0.6  0.4  -0.8  -0.7 -1.6   0.1   0.305  
3  13.6  24.8  2.3  2.0   2.9   0.8  3.7   3.0   0.305  
4  12.3  25.7  1.2  0.5   1.5  -1.4  0.1   0.6   0.305  

[5 rows x 33 columns]

In [87]:
test.to_csv(r"C:\Ziyuan Sui\Data Science Extra\My Own Project\All NBA Team Prediction\2024\test2024_final.csv",index=False)